In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
import os
os.environ["TF_ENABLE_ONEDNN_OPTS"]='0'

#NN-based model
from deepctr_torch.models import PNN
from deepctr_torch.models import CCPM

from deepctr_torch.models import WDL 
from deepctr_torch.models import DCN
from deepctr_torch.models import NFM
from deepctr_torch.models import DeepFM

#recent nn-based approach
from deepctr_torch.models import AFM
from deepctr_torch.models import xDeepFM
from deepctr_torch.inputs import SparseFeat,get_feature_names,DenseFeat
import torch


2023-05-02 19:39:26.894463: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-02 19:39:27.863710: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
torch.cuda.is_available()

True

In [3]:
embed_X = np.load("../hw3_data/Movielens/embedding.npy")
user_movie = np.load("../hw3_data/Movielens/user_movie.npy")
y = user_movie[:, 2]

sparse_feature_columns = ['C' + str(i) for i in range(1, embed_X.shape[1]+1)]
data = pd.DataFrame(embed_X,columns=sparse_feature_columns)
data['label'] = y
target = ['label']


In [4]:
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique(),embedding_dim=2)
                       for i,feat in enumerate(sparse_feature_columns)] 

In [5]:
dnn_feature_columns = fixlen_feature_columns 
linear_feature_columns = fixlen_feature_columns 

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [7]:
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name] for name in feature_names}

test_model_input = {name:test[name] for name in feature_names}


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = DeepFM(linear_feature_columns,dnn_feature_columns,task='regression',device=device)
model.compile("adam", "mse", metrics=['mse'] )


history = model.fit(train_model_input,train['label'].values,batch_size=256,epochs=10,verbose=2,validation_split=0.2)
pred_ans = model.predict(test_model_input, batch_size=256)

cuda
Train on 63707 samples, validate on 15927 samples, 249 steps per epoch
Epoch 1/10
825s - loss:  14.1756 - mse:  14.1679 - val_mse:  0.9775


KeyboardInterrupt: 

In [ ]:
825/60

False

# source code

In [40]:
data = pd.read_csv('./criteo_sample.txt')

sparse_features_clo = ['C' + str(i) for i in range(1, 27)]
dense_features = ['I' + str(i) for i in range(1, 14)]

data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0, )
target = ['label']

data
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
data
feat
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique(),embedding_dim=4)
                       for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1,)
                      for feat in dense_features]
fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(fixlen_feature_columns + fixlen_feature_columns)
feature_names
dnn_feature_columns
len(feature_names)
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name] for name in feature_names}

test_model_input = {name:test[name] for name in feature_names}
train_model_input